# Data wrangle omics studies and Summary_of_primary_data.csv

In [1]:
import pickle
import random
import pandas as pd
import numpy as np
import statistics as stats
import random
from matplotlib import pyplot as plt
import time
import re

# omics studies

In [2]:
#p = open("../../data/intermediate_data_02/matches", "rb")
matches = pd.read_pickle("../../data/intermediate_data_02/matches").drop_duplicates()
matches[:5]

,gene_id,included,source,total_comparisons
0,Cre01.g000350.t1.1,0,Hemme_2014,1
1,Cre01.g000850.t1.2,0,Hemme_2014,1
2,Cre01.g001100.t1.2,0,Hemme_2014,1
3,Cre01.g001300.t1.1,0,Hemme_2014,1
4,Cre01.g002300.t1.2,0,Hemme_2014,1


In [3]:
matches.columns = ['transcript_id', 'num_detected', 'source', 'num_sampled']
matches.num_sampled = float(1)
matches['annotation_version']='v5.5'
matches.num_detected = matches.num_detected.apply(lambda x: float(1) if x > 0 else float(0))
#matches.num_sampled = matches.num_sampled.apply(lambda x: float(x))
matches['num_manipulated'] = float(0)
##matches[:5]

# primary_data

In [4]:
primary_data = pd.read_excel("../../data/raw_data/Summary_of_primary_data.xlsx", 'Sheet1')
primary_data = primary_data[['Genus', 'Species','First_author','Publication_year','Type_of_study','Genes_sampled','Genes_manipulated', 'Genes_with_effect',"Genes_no_effect"]]
primary_data[:3]
subset = primary_data[primary_data.Genus=='Chlamydomonas'][primary_data.Species=='reinhardtii']
subset[:3]

/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,Genus,Species,First_author,Publication_year,Type_of_study,Genes_sampled,Genes_manipulated,Genes_with_effect,Genes_no_effect
1,Chlamydomonas,reinhardtii,Lv,2013,knock_out; transcriptomics,NaN,Cre05.g248150.t1; Cre06.g299050.t1.2,Cre06.g299050.t1.2,Cre05.g248150.t1
5,Chlamydomonas,reinhardtii,Miller,2010,transcriptomics,NaN,NaN,NaN,NaN
7,Chlamydomonas,reinhardtii,Kwak,2017,transcriptomics,NaN,NaN,NaN,NaN


In [5]:
for col in subset.columns[4:]:
    subset[col] = subset[col].apply(lambda x: tuple(x.split("; ")) if isinstance(x,str) else np.NaN) 
subset.head()

,Genus,Species,First_author,Publication_year,Type_of_study,Genes_sampled,Genes_manipulated,Genes_with_effect,Genes_no_effect
1,Chlamydomonas,reinhardtii,Lv,2013,"(knock_out, transcriptomics)",NaN,"(Cre05.g248150.t1, Cre06.g299050.t1.2)","(Cre06.g299050.t1.2,)","(Cre05.g248150.t1,)"
5,Chlamydomonas,reinhardtii,Miller,2010,"(transcriptomics,)",NaN,NaN,NaN,NaN
7,Chlamydomonas,reinhardtii,Kwak,2017,"(transcriptomics,)",NaN,NaN,NaN,NaN
8,Chlamydomonas,reinhardtii,Siaut,2011,"(knock_out,)",NaN,"(STA1, STA6, STA7, STA8)","(STA1, STA6, STA7, STA8)",NaN
9,Chlamydomonas,reinhardtii,Wang,2009,"(knock_out,)",NaN,"(STA6,)","(STA6,)",NaN


In [6]:
print(subset.columns)

Index(['Genus', 'Species', 'First_author', 'Publication_year', 'Type_of_study',
       'Genes_sampled', 'Genes_manipulated', 'Genes_with_effect',
       'Genes_no_effect'],
      dtype='object')


#### Convert `20201019_sampled_genes_key.csv` to dictionary format with `symbol` as keys and `transcript_id` as values

In [7]:
sampled_genes_key = pd.read_csv("../01_compile_raw_data/20201019_sampled_genes_key.csv")
print(sampled_genes_key.head())
#print()
key = dict(zip(sampled_genes_key.symbol, sampled_genes_key.transcript_id))
print(list(key.items())[:5])

                                              source symbol  \
0  Sim, L., Beeren, S. R., Findinier, J., Dauvill...   STA8   
1                                        Phytozome12   APL1   
2                                        Phytozome12  PEPC1   
3                                              Query   NRR1   
4                                              Query   SQD1   

        transcript_id  
0  Cre17.g698850.t1.1  
1  Cre16.g683450.t1.1  
2       Cre16.g673852  
3  Cre16.g673250.t1.1  
4  Cre16.g656400.t1.2  
[('CsnRK2', 'Cre10.g466350'), ('ACS2', 'Cre01.g055408.t1.1'), ('FEA1', 'Cre12.g546550.t1.1'), ('KAR', 'Cre03.g172000.t1.2'), ('SQD1', 'Cre16.g656400.t1.2')]


In [8]:
transformed_subset = pd.DataFrame(columns = ['transcript_id','num_sampled', 'num_manipulated', 'num_detected', 'source'])
for index, row in subset.iterrows():
    source = row.First_author + "_" + str(row.Publication_year)
    for i in range(0,3): 
        if type(row[i+5]) != float:
            for gene in row[i+5]: #Genes_sampled
                output = [0]*(i)+[1]+[0]*(2-i)
                try: 
                    gene = key[re.sub(r" |;","", gene)]
                except: gene = re.sub(r" |;","", gene)
                output = pd.Series([gene]+output+[source], index = transformed_subset.columns)
                transformed_subset = transformed_subset.append(output, ignore_index = True)
                #print(output)

In [9]:
transformed_subset['annotation_version'] = 'v5.5'

In [10]:
transformed_subset.head()

,transcript_id,num_sampled,num_manipulated,num_detected,source,annotation_version
0,Cre05.g248150.t1.2,0.0,1.0,0.0,Lv_2013,v5.5
1,Cre06.g299050.t1.2,0.0,1.0,0.0,Lv_2013,v5.5
2,Cre06.g299050.t1.2,0.0,0.0,1.0,Lv_2013,v5.5
3,Cre13.g567950.t1.2,0.0,1.0,0.0,Siaut_2011,v5.5
4,Cre03.g188250.t1.2,0.0,1.0,0.0,Siaut_2011,v5.5


### Merge matches and primary_data to one dataframe

In [11]:
merged = pd.concat([transformed_subset, matches], ignore_index=True)
grouped = matches.groupby(['transcript_id', 'source']).sum()
print(grouped[:3])

                                  num_detected  num_sampled  num_manipulated
transcript_id      source                                                   
Cre01.g000017.t1.1 Bajhaiya_2016           0.0          1.0              0.0
                   Kwak_2017               0.0          1.0              0.0
Cre01.g000033.t1.1 Bajhaiya_2016           1.0          1.0              0.0


In [12]:
transcript_list = [list(i)[0] for i in grouped.index]
source_list = [list(i)[1] for i in grouped.index]

df= pd.DataFrame.from_dict({"num_detected": grouped.num_detected,
                            "num_sampled": grouped.num_sampled,
                            'num_manipulated': grouped.num_manipulated}).reset_index()
df.head()

,transcript_id,source,num_detected,num_manipulated,num_sampled
0,Cre01.g000017.t1.1,Bajhaiya_2016,0.0,0.0,1.0
1,Cre01.g000017.t1.1,Kwak_2017,0.0,0.0,1.0
2,Cre01.g000033.t1.1,Bajhaiya_2016,1.0,0.0,1.0
3,Cre01.g000050.t1.1,Gargouri_2015,0.0,0.0,1.0
4,Cre01.g000100.t1.1,Bajhaiya_2016,0.0,0.0,1.0


In [13]:
#Check if any num_detected > num_sampled, because this shouldn't happen
df[df.num_detected>df.num_sampled]

,transcript_id,source,num_detected,num_manipulated,num_sampled


In [14]:
with open("../../data/intermediate_data_02/sampled_genes.pk", "wb") as f:
    pickle.dump(df, f)